<a href="https://colab.research.google.com/github/FatmaSayedAhmed/PhDResearch/blob/master/DR1_GPU_2Classes_DenseNet169_Pre_trained_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


### Class 0 Vs Class 4

In [2]:
import tensorflow as tf 
import keras
import os

from subprocess import check_output

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from PIL import Image
              
from keras.applications.densenet import preprocess_input     

from keras.preprocessing import image

from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

from sklearn.cross_validation import train_test_split

from keras.utils import np_utils

from keras.applications.densenet import DenseNet169   

from keras.models import Model, load_model

from keras.layers import Dense, Dropout, Reshape

from keras.optimizers import SGD

import matplotlib.pyplot as plt

from sklearn.metrics import classification_report,confusion_matrix

from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Flatten 
from keras import backend as K
from keras import applications


os.environ["CUDA_VISIBLE_DEVICES"] = '0' #use GPU with ID=0
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.7 # maximun alloc gpu50% of MEM
#config.gpu_options.allow_growth = True #allocate dynamically
sess = tf.Session(config = config)
keras.backend.set_session(sess)


# dimensions of our images.
img_width, img_height = 224, 224

top_model_weights_path = '/content/gdrive/My Drive/Colab Notebooks/bottleneck_fc_model_DenseNet169_PTN_data4.h5'

train_data_dir = '/content/gdrive/My Drive/Colab Notebooks/data4_Train496_Test212/train'
validation_data_dir = '/content/gdrive/My Drive/Colab Notebooks/data4_Train496_Test212/validation'
nb_train_samples = 992
nb_validation_samples = 424

epochs = 50
batch_size = 8  # batch size in flow_images_from_directory needs to correspond to the image number of the test images.


def save_bottleneck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the Inception_ResNet_V2 network

    model = applications.DenseNet169(weights='imagenet', include_top=False)

    #model.summary()
    
    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('DR1_bottleneck_features_train_DenseNet169_PTN_data4.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('DR1_bottleneck_features_validation_DenseNet169_PTN_data4.npy', 'wb'),
            bottleneck_features_validation)


def train_top_model():
    train_data = np.load(open('DR1_bottleneck_features_train_DenseNet169_PTN_data4.npy','rb'))
    train_labels = np.array(
        [0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

    print(' train_labels shape',train_labels.shape)

    #print(' train_labels : \n ',train_labels)

    validation_data = np.load(open('DR1_bottleneck_features_validation_DenseNet169_PTN_data4.npy','rb'))
    validation_labels = np.array(
        [0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

    print(' validation_labels shape',validation_labels.shape)

    #print(' validation_labels : \n ',validation_labels)

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)
    
    print('-------------------------------------------------------------------')
    print('------------------ Evalute the train set ------------------------')
    print('-------------------------------------------------------------------')

    train_score = model.evaluate(train_data, train_labels, verbose=0) # nb_train_samples // batch_size, workers=1)
    print('Train loss:', train_score[0])
    print('Train accuracy:', train_score[1])

    y_pred_train = np.squeeze(model.predict(train_data))
    print(' y_pred_train shape',y_pred_train.shape)
    #print(' y_pred_train : \n ',y_pred_train)

    threshold = 0.5
    y_pred_train = (y_pred_train > threshold)*1
    #y_pred_train.astype(int)   
    #print(' y_pred_train again: \n ',y_pred_train)
    
    accuracy_score_train = accuracy_score(train_labels, y_pred_train, normalize=True)
    print('accuracy_score_train with normalize=True: ', accuracy_score_train)

    accuracy_score_train = accuracy_score(train_labels, y_pred_train, normalize=False)
    print('accuracy_score_train with normalize=False : ', accuracy_score_train)

    target_names = ['class 0', 'class 1']

    print(classification_report(y_pred_train, train_labels, target_names=target_names))

    cm1 = confusion_matrix(y_pred_train, train_labels)
    
    print('confusion_matrix : \n', cm1)

    total1=sum(sum(cm1))

    #####from confusion matrix calculate accuracy
    accuracy1=(cm1[0,0]+cm1[1,1])/total1
    print ('Accuracy : ', accuracy1)

    sensitivity = cm1[0,0]/(cm1[0,0]+cm1[1,0])
    print('Sensitivity : ', sensitivity )

    Specificity = cm1[1,1]/(cm1[1,1]+cm1[0,1])
    print('Specificity : ', Specificity )

    print('-------------------------------------------------------------------')
    print('------------------ Evalute the validation set ------------------------')
    print('-------------------------------------------------------------------')

    validation_score = model.evaluate(validation_data, validation_labels, verbose=0) # nb_train_samples // batch_size, workers=1)
    print('validation loss:', validation_score[0])
    print('validation accuracy:', validation_score[1])

    y_pred_validation = np.squeeze(model.predict(validation_data))
    print(' y_pred_validation shape',y_pred_validation.shape)
    #print(' y_pred_validation : \n ',y_pred_validation)

    y_pred_validation = (y_pred_validation > threshold)*1
    #y_pred_train.astype(int)   
    #print(' y_pred_validation again: \n ',y_pred_validation)
    
    accuracy_score_validation = accuracy_score(validation_labels, y_pred_validation, normalize=True)
    print('accuracy_score_validation with normalize=True: ', accuracy_score_validation)

    accuracy_score_validation = accuracy_score(validation_labels, y_pred_validation, normalize=False)
    print('accuracy_score_validation with normalize=False : ', accuracy_score_validation)

    print(classification_report(y_pred_validation, validation_labels, target_names=target_names))

    cm1 = confusion_matrix(y_pred_validation, validation_labels)
    
    print('confusion_matrix : \n', cm1)

    total1=sum(sum(cm1))

    #####from confusion matrix calculate accuracy
    accuracy1=(cm1[0,0]+cm1[1,1])/total1
    print ('Accuracy : ', accuracy1)

    sensitivity = cm1[0,0]/(cm1[0,0]+cm1[1,0])
    print('Sensitivity : ', sensitivity )

    Specificity = cm1[1,1]/(cm1[1,1]+cm1[0,1])
    print('Specificity : ', Specificity )


save_bottleneck_features()
train_top_model()

print('-------------------------------------------------------------------')
print('------------------ Train Done -------------------------------------')
print('-------------------------------------------------------------------')

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


53182464/53178568 [==============================] - 1s 0us/step
Found 996 images belonging to 2 classes.
Found 424 images belonging to 2 classes.
 train_labels shape (992,)
 validation_labels shape (424,)
Train on 992 samples, validate on 424 samples
Epoch 1/50
992/992 [==============================] - 7s 7ms/step - loss: 7.7705 - acc: 0.5121 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 2/50
992/992 [==============================] - 4s 4ms/step - loss: 6.5790 - acc: 0.5827 - val_loss: 5.1917 - val_acc: 0.6675
Epoch 3/50
992/992 [==============================] - 4s 4ms/step - loss: 4.5784 - acc: 0.7006 - val_loss: 4.9638 - val_acc: 0.6769
Epoch 4/50
992/992 [==============================] - 4s 4ms/step - loss: 4.7473 - acc: 0.6986 - val_loss: 6.1284 - val_acc: 0.6108
Epoch 5/50
992/992 [==============================] - 4s 4ms/step - loss: 4.9534 - acc: 0.6845 - val_loss: 4.6693 - val_acc: 0.6958
Epoch 6/50
992/992 [==============================] - 4s 4ms/step - loss: 4.2037 - acc: 